In [3]:
import time
import sqlite3
from datetime import datetime

# ==============================
# GPIO / MOCK SETUP
# ==============================
MOCK_MODE = False

try:
    import RPi.GPIO as GPIO
except ImportError:
    # Not running on Raspberry Pi → use a mock GPIO for testing (Colab/PC)
    MOCK_MODE = True

    class MockGPIO:
        BCM = 0
        IN = 0
        OUT = 0
        HIGH = 1
        LOW = 0

        def setmode(self, *args, **kwargs):
            print("[MOCK GPIO] setmode called")

        def setup(self, *args, **kwargs):
            print(f"[MOCK GPIO] setup called with args={args}")

        def input(self, pin):
            """
            Return fixed or test values for pins.
            You can change this logic to simulate different conditions.
            For now:
                rain = 0  (no rain)
                soil = 1  (dry)
                water_level = 1 (enough water)
            """
            if pin == 17:   # RAIN_SENSOR_PIN
                return 0    # no rain
            elif pin == 27: # SOIL_MOISTURE_PIN
                return 1    # dry
            elif pin == 22: # WATER_LEVEL_PIN
                return 1    # enough water
            return 0

        def output(self, pin, value):
            print(f"[MOCK GPIO] Pin {pin} set to {value}")

        def cleanup(self):
            print("[MOCK GPIO] cleanup called")

    GPIO = MockGPIO()

# ==============================
# GPIO PIN CONFIGURATION (BCM)
# ==============================
# CHANGE THESE PINS FOR YOUR REAL WIRING ON RASPBERRY PI
RAIN_SENSOR_PIN = 17          # Rain sensor digital output
SOIL_MOISTURE_PIN = 27        # Soil moisture digital output
WATER_LEVEL_PIN = 22          # Water level sensor / float switch

RELAY_PUMP_PIN = 23           # Relay channel to pump
RELAY_VALVE_PIN = 24          # Relay channel to solenoid valve

# ==============================
# LOGIC / THRESHOLDS
# ==============================
# For typical digital soil sensor: 1 = DRY, 0 = WET (verify yours)
SOIL_DRY = 1

# For typical float sensor (depends on wiring):
#   0 = TANK LOW, 1 = ENOUGH WATER
TANK_LOW = 0

# For testing in Colab, keep this small. On Pi you can increase.
POLL_INTERVAL = 2            # seconds between each reading
DB_PATH = "water_harvesting.db"


# ==============================
# SETUP FUNCTIONS
# ==============================
def setup_gpio():
    GPIO.setmode(GPIO.BCM)

    # Sensor pins as input
    GPIO.setup(RAIN_SENSOR_PIN, GPIO.IN)
    GPIO.setup(SOIL_MOISTURE_PIN, GPIO.IN)
    GPIO.setup(WATER_LEVEL_PIN, GPIO.IN)

    # Relay pins as output
    GPIO.setup(RELAY_PUMP_PIN, GPIO.OUT)
    GPIO.setup(RELAY_VALVE_PIN, GPIO.OUT)

    # Assume relay board is ACTIVE LOW:
    # HIGH = OFF, LOW = ON
    GPIO.output(RELAY_PUMP_PIN, GPIO.HIGH)
    GPIO.output(RELAY_VALVE_PIN, GPIO.HIGH)


def init_db():
    """Create SQLite database + table if not already present."""
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        """
        CREATE TABLE IF NOT EXISTS sensor_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT NOT NULL,
            rain INTEGER NOT NULL,
            soil_moisture INTEGER NOT NULL,
            water_level INTEGER NOT NULL,
            pump_on INTEGER NOT NULL,
            valve_on INTEGER NOT NULL
        )
        """
    )
    conn.commit()
    conn.close()


def log_data(rain, soil, water_level, pump_on, valve_on):
    """Insert one row of sensor + actuator status into DB."""
    # Ensure everything is basic Python int
    rain = int(rain)
    soil = int(soil)
    water_level = int(water_level)
    pump_on = int(bool(pump_on))
    valve_on = int(bool(valve_on))

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        """
        INSERT INTO sensor_logs
            (timestamp, rain, soil_moisture, water_level, pump_on, valve_on)
        VALUES (?, ?, ?, ?, ?, ?)
        """,
        (
            datetime.now().isoformat(timespec="seconds"),
            rain,
            soil,
            water_level,
            pump_on,
            valve_on,
        ),
    )
    conn.commit()
    conn.close()


# ==============================
# SENSOR READ FUNCTIONS
# ==============================
def read_rain_sensor():
    """
    Returns:
        1 if rain detected
        0 if no rain
    """
    return int(GPIO.input(RAIN_SENSOR_PIN))


def read_soil_moisture():
    """
    Digital soil sensor typical behavior:
        1 -> dry
        0 -> wet
    """
    return int(GPIO.input(SOIL_MOISTURE_PIN))


def read_water_level():
    """
    Water level float sensor:
        0 -> low level (tank nearly empty)
        1 -> enough water
    """
    return int(GPIO.input(WATER_LEVEL_PIN))


# ==============================
# ACTUATOR CONTROL
# ==============================
def set_pump(on: bool):
    """Turn pump relay ON/OFF (active LOW)."""
    GPIO.output(RELAY_PUMP_PIN, GPIO.LOW if on else GPIO.HIGH)


def set_valve(on: bool):
    """Turn solenoid valve relay ON/OFF (active LOW)."""
    GPIO.output(RELAY_VALVE_PIN, GPIO.LOW if on else GPIO.HIGH)


# ==============================
# CONTROL LOGIC
# ==============================
def control_logic(rain, soil, water_level):
    """
    Simple rule-based control:

    - If soil is DRY and tank has ENOUGH water:
          -> Pump ON, Valve OPEN (irrigate plants).
    - Else:
          -> Pump OFF, Valve CLOSED.
    """
    pump_on = False
    valve_on = False

    soil_is_dry = (soil == SOIL_DRY)
    tank_is_low = (water_level == TANK_LOW)

    if soil_is_dry and not tank_is_low:
        pump_on = True
        valve_on = True
    else:
        pump_on = False
        valve_on = False

    # Apply to hardware
    set_pump(pump_on)
    set_valve(valve_on)

    return pump_on, valve_on


# ==============================
# MAIN LOOP
# ==============================
def main():
    print("Starting Raspberry Pi Water Harvesting System...")

    setup_gpio()
    init_db()

    # In MOCK mode (Colab/PC): run only a fixed number of iterations
    if MOCK_MODE:
        print("Running in MOCK mode (Colab / PC) – GPIO is simulated.")
        max_iterations = 5   # stop after 5 cycles
    else:
        print("Running on Raspberry Pi – GPIO is real.")
        max_iterations = None  # run forever on real Pi

    try:
        iteration = 0

        while True:
            rain = read_rain_sensor()
            soil = read_soil_moisture()
            water_level = read_water_level()

            pump_on, valve_on = control_logic(rain, soil, water_level)

            # Log to database
            log_data(rain, soil, water_level, pump_on, valve_on)

            # Print status
            print(
                f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] "
                f"Rain={rain}, Soil={soil}, WaterLevel={water_level}, "
                f"Pump={'ON' if pump_on else 'OFF'}, "
                f"Valve={'ON' if valve_on else 'OFF'}"
            )

            iteration += 1

            # Stop after some loops in MOCK mode
            if max_iterations is not None and iteration >= max_iterations:
                print("MOCK mode test complete. Stopping program.")
                break

            time.sleep(POLL_INTERVAL)

    except KeyboardInterrupt:
        print("\nShutting down system...")

    finally:
        GPIO.cleanup()


if __name__ == "__main__":
    main()


Starting Raspberry Pi Water Harvesting System...
[MOCK GPIO] setmode called
[MOCK GPIO] setup called with args=(17, 0)
[MOCK GPIO] setup called with args=(27, 0)
[MOCK GPIO] setup called with args=(22, 0)
[MOCK GPIO] setup called with args=(23, 0)
[MOCK GPIO] setup called with args=(24, 0)
[MOCK GPIO] Pin 23 set to 1
[MOCK GPIO] Pin 24 set to 1
Running in MOCK mode (Colab / PC) – GPIO is simulated.
[MOCK GPIO] Pin 23 set to 0
[MOCK GPIO] Pin 24 set to 0
[2025-12-04 13:26:17] Rain=0, Soil=1, WaterLevel=1, Pump=ON, Valve=ON
[MOCK GPIO] Pin 23 set to 0
[MOCK GPIO] Pin 24 set to 0
[2025-12-04 13:26:19] Rain=0, Soil=1, WaterLevel=1, Pump=ON, Valve=ON
[MOCK GPIO] Pin 23 set to 0
[MOCK GPIO] Pin 24 set to 0
[2025-12-04 13:26:21] Rain=0, Soil=1, WaterLevel=1, Pump=ON, Valve=ON
[MOCK GPIO] Pin 23 set to 0
[MOCK GPIO] Pin 24 set to 0
[2025-12-04 13:26:23] Rain=0, Soil=1, WaterLevel=1, Pump=ON, Valve=ON
[MOCK GPIO] Pin 23 set to 0
[MOCK GPIO] Pin 24 set to 0
[2025-12-04 13:26:25] Rain=0, Soil=1, 